Gather all tickers

In [93]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from typing import List, Tuple

wiki_page = requests.get('https://en.wikipedia.org/wiki/List_of_American_exchange-traded_funds').text
soup = BeautifulSoup(wiki_page, 'lxml')

list_items = soup.select('li:contains("|")')
tickers = []

for list_item in list_items:
    li_text: str = list_item.text
    start_index: int = li_text.find('|')
    end_index: int = li_text.find(')',start_index)
    tickers.append(li_text[start_index +1:end_index].strip())

print(tickers)

['ITOT', 'ACWI', 'IWV', 'SCHB', 'FNDB', 'VT', 'VTI', 'VXUS', 'VTHR', 'DIA', 'RSP', 'IOO', 'IVV', 'SPY', 'SHE', 'VOO', 'IWM', 'OEF', 'QQQ', 'CVY', 'RPG', 'RPV', 'IWB', 'IWF', 'IWD', 'IVV', 'IVW', 'IVE', 'PKW', 'PRF', 'SPLV', 'SCHX', 'SCHG', 'SCHV', 'SCHD', 'FNDX', 'SDY', 'VOO', 'VOOG', 'VOOV', 'VV', 'VUG', 'VTV', 'MGC', 'MGK', 'MGV', 'VONE', 'VONG', 'VONV', 'VIG', 'VYM', 'DTN', 'DLN', 'MDY', 'DVY', 'IWR', 'IWP', 'IWS', 'IJH', 'IJK', 'IJJ', 'PDP', 'SCHM', 'IVOO', 'IVOG', 'IVOV', 'VO', 'VOT', 'VOE', 'VXF', 'DON', 'IWC', 'IWM', 'IWO', 'IWN', 'IJR', 'IJT', 'IJS', 'SCHA', 'FNDA', 'VIOO', 'VIOG', 'VIOV', 'VB', 'VBK', 'VBR', 'VTWO', 'VTWG', 'VTWV', 'EEB', 'ECON', 'IDV', 'ACWX', 'BKF', 'EFA', 'EFG', 'EFV', 'SCZ', 'EEM', 'PID', 'SCHC', 'SCHE', 'SCHF', 'FNDF', 'FNDC', 'FNDE', 'DWX', 'VEA', 'VWO', 'VXUS', 'VEU', 'VSS', 'DEM', 'DGS', 'AAXJ', 'EZU', 'EPP', 'IEV', 'ILF', 'FEZ', 'VGK', 'VPL', 'HEDJ', 'DFE', 'AND', 'GXF', 'EWA', 'EWC', 'EWG', 'EIS', 'EWI', 'EWJ', 'EWD', 'EWL', 'EWP', 'EWU', 'DXJ', 'NOR

In [94]:
leveraged_page = requests.get('https://etfdb.com/etfs/leveraged/equity/').text
soup = BeautifulSoup(leveraged_page, 'lxml')

list_items = soup.select('td[data-th="Symbol"] > a')
for list_item in list_items:
    tickers.append(list_item.text)
    
print(tickers)

['ITOT', 'ACWI', 'IWV', 'SCHB', 'FNDB', 'VT', 'VTI', 'VXUS', 'VTHR', 'DIA', 'RSP', 'IOO', 'IVV', 'SPY', 'SHE', 'VOO', 'IWM', 'OEF', 'QQQ', 'CVY', 'RPG', 'RPV', 'IWB', 'IWF', 'IWD', 'IVV', 'IVW', 'IVE', 'PKW', 'PRF', 'SPLV', 'SCHX', 'SCHG', 'SCHV', 'SCHD', 'FNDX', 'SDY', 'VOO', 'VOOG', 'VOOV', 'VV', 'VUG', 'VTV', 'MGC', 'MGK', 'MGV', 'VONE', 'VONG', 'VONV', 'VIG', 'VYM', 'DTN', 'DLN', 'MDY', 'DVY', 'IWR', 'IWP', 'IWS', 'IJH', 'IJK', 'IJJ', 'PDP', 'SCHM', 'IVOO', 'IVOG', 'IVOV', 'VO', 'VOT', 'VOE', 'VXF', 'DON', 'IWC', 'IWM', 'IWO', 'IWN', 'IJR', 'IJT', 'IJS', 'SCHA', 'FNDA', 'VIOO', 'VIOG', 'VIOV', 'VB', 'VBK', 'VBR', 'VTWO', 'VTWG', 'VTWV', 'EEB', 'ECON', 'IDV', 'ACWX', 'BKF', 'EFA', 'EFG', 'EFV', 'SCZ', 'EEM', 'PID', 'SCHC', 'SCHE', 'SCHF', 'FNDF', 'FNDC', 'FNDE', 'DWX', 'VEA', 'VWO', 'VXUS', 'VEU', 'VSS', 'DEM', 'DGS', 'AAXJ', 'EZU', 'EPP', 'IEV', 'ILF', 'FEZ', 'VGK', 'VPL', 'HEDJ', 'DFE', 'AND', 'GXF', 'EWA', 'EWC', 'EWG', 'EIS', 'EWI', 'EWJ', 'EWD', 'EWL', 'EWP', 'EWU', 'DXJ', 'NOR

In [95]:
benchmark = '^GSPC' # add S&P 500 as benchmark
tickers.append(benchmark) # append benchmark

Fetch ticker info from Yahoo

In [96]:
import yfinance as yf

data: pd.DataFrame = yf.download(tickers=" ".join(tickers), period="5y", interval="1d", group_by='ticker')
print(data)

[*********************100%***********************]  442 of 442 completed

21 Failed downloads:
- IRV: No data found for this date range, symbol may be delisted
- IFAS: No data found for this date range, symbol may be delisted
- GGBP: No data found for this date range, symbol may be delisted
- ONEF: No data found for this date range, symbol may be delisted
- ACCU: No data found for this date range, symbol may be delisted
- YPRO: No data found, symbol may be delisted
- FTGS: No data found, symbol may be delisted
- RPX: No data found, symbol may be delisted
- DBIZ: No data found for this date range, symbol may be delisted
- WDTI: No data found, symbol may be delisted
- RRF: No data found, symbol may be delisted
- QEH: No data found, symbol may be delisted
- IELG: No data found for this date range, symbol may be delisted
- BGU: No data found for this date range, symbol may be delisted
- HDGI: No data found for this date range, symbol may be delisted
- GLDE: No data found for this date rang

Delete existing file cache

In [117]:
import glob

files = glob.glob(os.path.join("data", '*'))

for file in files:
    os.remove(file)

Save output to file to prevent further network requests.

In [118]:
import os

found_tickers: List[str] = data.columns.get_level_values(0).unique().to_list()

for found_ticker in found_tickers:
    data[found_ticker].to_csv(os.path.join("data", found_ticker + '.csv'))


Read files back from directory.

In [124]:
csv_paths: List[str] = glob.glob(os.path.join("data", '*.csv'))
prices_df: pd.DataFrame = None

for csv_path in csv_paths:
    (ticker_id, extension) = csv_path.split(".", 1)
    df: pd.DataFrame = pd.read_csv(csv_path, index_col='Date', usecols=['Date', 'Adj Close'], header=0, parse_dates=True)
    df = df.rename(columns={'Adj Close': os.path.split(ticker_id)[1]})

    if prices_df is not None:
        prices_df = prices_df.join(df)
    else:
        prices_df = df

prices_df = prices_df.sort_values(by='Date', axis=0)
prices_df

,AADR,AAXJ,ACCU,ACWI,ACWX,AGG,ALD,AMLP,AND,ARGT,...,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XOP,YPRO,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
2015-12-21,36.689663,49.088512,NaN,49.806164,34.482517,95.325974,40.378086,35.828045,6.043314,16.845085,...,13.364078,47.330025,39.670265,43.935196,36.271324,65.498077,72.790520,107.474060,NaN,2021.150024
2015-12-22,37.496986,49.335468,NaN,50.193382,34.745468,95.255585,40.368687,37.308540,5.956706,17.286360,...,13.489031,47.983608,39.958939,44.507500,36.578716,65.940094,73.324364,108.787476,NaN,2038.969971
2015-12-23,37.817982,49.966568,NaN,50.913780,35.280151,95.237961,40.707142,39.447037,6.206906,17.574142,...,13.648057,48.546410,40.294182,44.833275,37.142254,66.649117,73.698997,117.380913,NaN,2064.290039
2015-12-24,37.940552,49.829376,NaN,50.850731,35.183731,95.283829,40.537903,39.282536,6.312760,17.622110,...,13.636699,48.528259,40.275558,44.789257,37.116631,66.713577,73.502304,116.517799,NaN,2060.989990
2015-12-28,37.356850,49.399494,NaN,50.697659,35.052258,95.283829,40.246487,38.558735,6.158791,17.602922,...,13.625340,48.419327,40.247627,44.727612,37.184940,66.400490,73.689621,112.202332,NaN,2056.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-15,58.980000,87.230003,NaN,89.330002,52.340000,117.904953,NaN,27.840000,NaN,30.070000,...,28.610001,88.839996,126.830002,67.220001,63.139999,112.040001,158.639999,61.230000,NaN,3694.620117
2020-12-16,59.049999,87.510002,NaN,89.550003,52.529999,117.855003,NaN,27.430000,NaN,30.299999,...,28.660000,88.290001,127.690002,67.349998,62.410000,111.709999,159.679993,60.410000,NaN,3701.169922
2020-12-17,59.939999,88.070000,NaN,90.190002,52.919998,117.849998,NaN,27.219999,NaN,30.690001,...,28.740000,88.830002,128.699997,67.779999,62.830002,112.910004,160.770004,60.410000,NaN,3722.479980


In [125]:
most_recent_prices_df = prices_df.iloc[-1]
tickers_to_drop = most_recent_prices_df[most_recent_prices_df.isnull()].index.values
tickers_to_drop

array(['AADR', 'ACCU', 'ALD', 'AND', 'AUNZ', 'BABZ', 'BGU', 'BGZ', 'BRAF',
       'BRAQ', 'BRAZ', 'BZF', 'CIU', 'CRDT', 'CSJ', 'DBIZ', 'DI', 'EEB',
       'EPRO', 'EU', 'FORX', 'FTGS', 'FTLB', 'FWDB', 'FWDD', 'FWDI',
       'GEUR', 'GGBP', 'GIVE', 'GLDE', 'GMMB', 'GMTB', 'GTAA', 'GVT',
       'GYEN', 'HDGI', 'HECO', 'HOLD', 'HUSE', 'ICB', 'IEIL', 'IEIS',
       'IELG', 'IESM', 'IFAS', 'IFEU', 'ILB', 'INDF', 'INKM', 'IPD',
       'IPF', 'IPK', 'IPN', 'IPS', 'IPU', 'IPW', 'IRV', 'IRY', 'IST',
       'JJA', 'JJM', 'LALT', 'MATH', 'MINC', 'ONEF', 'QEH', 'RPX', 'RRF',
       'RWG', 'SCPB', 'SYE', 'SYG', 'SYV', 'TTFS', 'VEGA', 'WDTI', 'WREI',
       'YPRO'], dtype=object)

Calculate price returns

In [127]:
returns_df = prices_df.pct_change()
returns_df = returns_df.drop(columns=tickers_to_drop)

found_tickers = returns_df.columns
returns_df

,AAXJ,ACWI,ACWX,AGG,AMLP,ARGT,ARKG,ARKK,ARKQ,ARKW,...,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XOP,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
2015-12-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-12-22,0.005031,0.007775,0.007626,-0.000738,0.041322,0.026196,NaN,0.004312,NaN,0.005988,...,0.011725,0.009350,0.013809,0.007277,0.013026,0.008475,0.006749,0.007334,0.012221,0.008817
2015-12-23,0.012792,0.014352,0.015389,-0.000185,0.057319,0.016648,NaN,0.014647,NaN,0.008121,...,0.043500,0.011789,0.011729,0.008390,0.007320,0.015406,0.010753,0.005109,0.078993,0.012418
2015-12-24,-0.002746,-0.001238,-0.002733,0.000482,-0.004170,0.002729,NaN,0.003702,NaN,0.004702,...,-0.009014,-0.000832,-0.000374,-0.000462,-0.000982,-0.000690,0.000967,-0.002669,-0.007353,-0.001599
2015-12-28,-0.008627,-0.003010,-0.003737,0.000000,-0.018426,-0.001089,NaN,0.001485,NaN,-0.007557,...,-0.018191,-0.000833,-0.002245,-0.000694,-0.001376,0.001840,-0.004693,0.002548,-0.037037,-0.002179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-15,0.008789,0.012697,0.010815,0.000593,0.022777,0.019668,-0.010038,0.000402,0.015307,0.001870,...,0.019657,0.017064,0.013924,0.016673,0.002386,0.019703,0.010826,0.013480,0.021351,0.012921
2020-12-16,0.003210,0.002463,0.003630,-0.000424,-0.014727,0.007649,-0.014364,-0.002816,-0.004426,0.006082,...,-0.004943,0.001748,-0.006191,0.006781,0.001934,-0.011562,-0.002945,0.006556,-0.013392,0.001773
2020-12-17,0.006399,0.007147,0.007424,-0.000042,-0.007656,0.012871,0.008251,0.017104,0.020144,0.026723,...,-0.004719,0.002791,0.006116,0.007910,0.006385,0.006730,0.010742,0.006826,0.000000,0.005758


Calucate expected return using geomean from price return

In [128]:
from scipy.stats import gmean

exp_return_df = pd.DataFrame()
for found_ticker in found_tickers:
    returns_sr = returns_df[pd.notnull(returns_df[found_ticker])][found_ticker]
    if exp_return_df.empty:
        exp_return_df = pd.DataFrame(data={
            'ticker': found_ticker,
            'exp_return': [0] if returns_sr.empty else [gmean(returns_sr + 1) - 1]
        })
    else:
        exp_return_df = pd.concat([
            exp_return_df,
            pd.DataFrame(data={
                'ticker': found_ticker,
                'exp_return': [0] if returns_sr.empty else [gmean(returns_sr + 1) - 1]
            })
        ])

exp_return_df = exp_return_df.set_index('ticker').T
exp_return_df

ticker,AAXJ,ACWI,ACWX,AGG,AMLP,ARGT,ARKG,ARKK,ARKQ,ARKW,...,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XOP,^GSPC
exp_return,0.000449,0.000452,0.000317,0.000168,-0.000248,0.000462,0.004187,0.001503,0.002762,0.001608,...,-0.000192,0.000594,0.000479,0.000915,0.000324,0.000406,0.000409,0.000608,-0.000497,0.000467


Calculate expected covariance using price return

In [129]:
covar_df = returns_df.cov()
covar_df

,AAXJ,ACWI,ACWX,AGG,AMLP,ARGT,ARKG,ARKK,ARKQ,ARKW,...,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XOP,^GSPC
AAXJ,0.000167,0.000130,0.000139,1.966110e-06,0.000144,0.000158,0.000416,0.000174,0.000403,0.000166,...,0.000166,0.000141,1.308275e-04,1.510009e-04,0.000073,0.000072,1.013638e-04,0.000125,0.000200,1.248312e-04
ACWI,0.000130,0.000134,0.000132,1.343716e-06,0.000155,0.000151,0.000464,0.000174,0.000462,0.000163,...,0.000179,0.000160,1.460620e-04,1.577034e-04,0.000089,0.000092,1.144642e-04,0.000134,0.000214,1.369576e-04
ACWX,0.000139,0.000132,0.000140,2.878750e-06,0.000162,0.000156,0.000447,0.000164,0.000438,0.000153,...,0.000181,0.000155,1.399892e-04,1.448638e-04,0.000081,0.000083,1.042491e-04,0.000126,0.000217,1.279012e-04
AGG,0.000002,0.000001,0.000003,8.295402e-06,0.000008,0.000005,0.000036,0.000005,0.000031,0.000005,...,0.000003,-0.000003,1.468809e-07,8.084259e-07,0.000002,0.000005,-9.328517e-07,0.000002,-0.000003,5.406104e-07
AMLP,0.000144,0.000155,0.000162,8.358244e-06,0.000515,0.000209,0.000603,0.000207,0.000615,0.000184,...,0.000363,0.000217,1.857731e-04,1.572228e-04,0.000081,0.000082,1.173101e-04,0.000153,0.000502,1.567386e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XLU,0.000072,0.000092,0.000083,4.993972e-06,0.000082,0.000078,0.000393,0.000083,0.000379,0.000076,...,0.000118,0.000110,1.059726e-04,1.022903e-04,0.000102,0.000174,9.008236e-05,0.000087,0.000102,1.008398e-04
XLV,0.000101,0.000114,0.000104,-9.328517e-07,0.000117,0.000118,0.000415,0.000155,0.000372,0.000136,...,0.000141,0.000135,1.256235e-04,1.399941e-04,0.000087,0.000090,1.374742e-04,0.000113,0.000160,1.249571e-04
XLY,0.000125,0.000134,0.000126,2.350296e-06,0.000153,0.000151,0.000481,0.000193,0.000489,0.000185,...,0.000172,0.000160,1.498293e-04,1.666891e-04,0.000089,0.000087,1.133043e-04,0.000158,0.000208,1.424959e-04
XOP,0.000200,0.000214,0.000217,-2.601134e-06,0.000502,0.000283,0.000626,0.000257,0.000714,0.000228,...,0.000540,0.000304,2.655368e-04,2.195544e-04,0.000112,0.000102,1.598097e-04,0.000208,0.000850,2.192199e-04


Reorder expected return to line up with covar matrix

In [130]:
exp_return_df = exp_return_df[covar_df.columns]
exp_return_df

ticker,AAXJ,ACWI,ACWX,AGG,AMLP,ARGT,ARKG,ARKK,ARKQ,ARKW,...,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XOP,^GSPC
exp_return,0.000449,0.000452,0.000317,0.000168,-0.000248,0.000462,0.004187,0.001503,0.002762,0.001608,...,-0.000192,0.000594,0.000479,0.000915,0.000324,0.000406,0.000409,0.000608,-0.000497,0.000467


Optimization of portfolio

In [131]:
alpha_df = exp_return_df.T - exp_return_df[benchmark] # subtract market return
alpha_df = alpha_df[alpha_df['exp_return'] > 0] # filter to only positive alpha
alpha_sr = alpha_df['exp_return'].to_numpy()

covar_df = covar_df[covar_df.index.isin(alpha_df.index)][alpha_df.index] # filter to only positive alpha
covar = covar_df.round(8).to_numpy()
alpha_df

,exp_return
ticker,
ARKG,0.003720
ARKK,0.001036
ARKQ,0.002295
ARKW,0.001141
BKF,0.000040
...,...
XLB,0.000003
XLF,0.000127
XLI,0.000012


In [132]:
from scipy.optimize import minimize
import numpy as np

def treynor_ratio(weights: np.ndarray, covar_matrix: np.ndarray, alpha_returns: np.ndarray) -> float:
    # we are minimizing the negative to get a maximum
    objective = float(-weights.dot(alpha_returns) / np.sqrt(weights.dot(covar_matrix).dot(weights.T)))
    return objective

weights = np.ones(covar_df.columns.__len__())
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})
# bounds are in the form of (lower, upper)
bounds = [(0, None,) for i in range(len(weights))] # bounded by zero (no shorting)
portfolio = minimize(treynor_ratio, weights, args=(covar, alpha_sr), bounds=bounds, constraints=constraints)
portfolio

     fun: -0.1162960897801975
     jac: array([-8.20495188e-07,  7.13222027e-02,  1.38459727e-05,  4.95045530e-02,
        4.98576825e-02,  4.35912637e-02,  4.18366874e-02,  1.01812303e-01,
        5.57204187e-02,  2.43049320e-02,  3.94276818e-02,  8.09703907e-02,
        1.60679467e-01,  1.03268583e-01,  5.32210078e-02,  9.64907082e-02,
        1.32082670e-01,  3.71745862e-02,  5.88213373e-02,  5.20112487e-02,
        6.38555624e-02,  6.83414396e-02,  5.42621920e-02,  7.91399125e-02,
        5.49252676e-02,  5.72137004e-02,  5.48320934e-02,  5.35761984e-02,
        5.51473722e-02,  5.42029962e-02,  6.63334858e-02,  6.85179634e-02,
        5.95670827e-02,  6.10845424e-02,  5.68350917e-02,  5.51445922e-02,
        5.10818735e-02,  5.50608039e-02,  5.38806403e-02,  5.40847890e-02,
        5.52987801e-02,  1.93582427e-01,  5.47399670e-02,  1.42066078e-02,
        5.33528458e-02,  2.95258882e-02,  6.25195066e-02,  6.40135994e-02,
        9.66027109e-02,  5.13234995e-02,  9.63346893e-02,  6

In [133]:
# send weights to clipboard
pd.Series(portfolio.x).to_clipboard()

In [134]:
# send expected alphas to clipboard
alpha_df.to_clipboard()

In [135]:
# send covars to clipboard
covar_df.to_clipboard()